**1. Upload the video file to data folder**

In [142]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [143]:
# Відкриваємо файли
cap = cv2.VideoCapture("data/Match.mp4")
template = cv2.imread("data/template.png", 0)
h, w = template.shape[:2]

In [144]:
output_size = (1080, 1920)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # або 'XVID' для .avi
out = cv2.VideoWriter('data/output.mp4', fourcc, 30.0, output_size)

In [145]:
# 4 опорні точки з кадру з відео)
pts_src = np.array([[210, 60], [945, 60], [1113, 1240], [114, 1240]], dtype=np.float32)

# відповідні точки на "випрямленому" зображенні
pts_dst = np.array([[0, 0], [1080, 0], [1080, 1920], [0, 1920]], dtype=np.float32)

# Матриця гомографії для вирівнювання перспективи
H, _ = cv2.findHomography(pts_src, pts_dst)

In [146]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # перетворення перспективи до кадру
    warped = cv2.warpPerspective(frame, H, (1080, 1920))
    gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY) # у сірий

    # шукаю шаблон на кадрі
    res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # якщо результат співпадіння перевищує трешхолд — об'єкт знайдено
    threshold = 0.83
    if max_val >= threshold:
        top_left = max_loc
        h, w = template.shape
        bottom_right = (top_left[0] + w, top_left[1] + h)

        # малюю прямокутник навколо знайденого об'єкта
        cv2.rectangle(warped, top_left, bottom_right, (255, 0, 0), 2)
        
        # підписую знайдений об'єкт
        cv2.putText(warped, f"{max_val:.3f}", (top_left[0], top_left[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Записуємо оброблений кадр у відео
    out.write(warped)
    
    # зменшую розмір кадру в 2 рази для зручного перегляду
    scaled_warped = cv2.resize(warped, (0, 0), fx=1, fy=1)
    cv2.imshow("Warped View", scaled_warped)

    # вихід по клавіші Esc
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()